In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D

In [ ]:
from keras.datasets import cifar10
# let's load data 
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
#normalizing inputs from 0-255 to 0.0-1.0 
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 
X_train = X_train / 255.0 
X_test = X_test / 255.0

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

In [ ]:
# one hot encode outputs 
from keras.utils import np_utils 

y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 
num_classes = y_test.shape[1]

In [ ]:
import keras
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras import backend as K 

In [ ]:
model = Sequential()

# 2D concolutional layer.32 == filters needed.(an array of numeric values). (3,3)==size of the filter(3 r, 3 c)
# 32,32,3 - shape of data-h,w,rgb values(depth/intensity)
# relu activation replaces -ve pixel values with 0s.
model.add(Conv2D(32, (3, 3), input_shape=(32,32,3), activation='relu', padding='same'))  #output=feature maps
# dropout used to prevent overfitting
model.add(Dropout(0.2)) 
model.add(Conv2D(32, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
# maxpooling==dim reduction - while retaining most important features
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model.add(Dropout(0.2)) 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model.add(Dropout(0.2)) 
model.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
# flatten used to convert feature map to 1D.
model.add(Flatten())
model.add(Dropout(0.2)) 
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3))) 
model.add(Dropout(0.2)) 
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3))) 
model.add(Dropout(0.2)) 
# softmax used - multiclass classification
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        3

In [ ]:
# model compilation
learning_rate = 0.01
epochs = 50 # we want a high quality model 
decay = learning_rate/epochs 
sgd = SGD(lr=learning_rate, momentum=0.9, decay=decay, nesterov=False) #optimisation function - stochastic gradient descent
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# model training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32) 

Epoch 1/50
1563/1563 [==============================] - 473s 303ms/step - loss: 1.8525 - accuracy: 0.3158 - val_loss: 1.4747 - val_accuracy: 0.4601
Epoch 2/50
1563/1563 [==============================] - 474s 303ms/step - loss: 1.4178 - accuracy: 0.4814 - val_loss: 1.2903 - val_accuracy: 0.5312
Epoch 3/50
1563/1563 [==============================] - 483s 309ms/step - loss: 1.2112 - accuracy: 0.5636 - val_loss: 1.1237 - val_accuracy: 0.5953
Epoch 4/50
1563/1563 [==============================] - 477s 305ms/step - loss: 1.0450 - accuracy: 0.6257 - val_loss: 0.9380 - val_accuracy: 0.6688
Epoch 5/50
1563/1563 [==============================] - 477s 305ms/step - loss: 0.9106 - accuracy: 0.6765 - val_loss: 0.8798 - val_accuracy: 0.6874
Epoch 6/50
1563/1563 [==============================] - 479s 306ms/step - loss: 0.8177 - accuracy: 0.7116 - val_loss: 0.7810 - val_accuracy: 0.7263
Epoch 7/50
1563/1563 [==============================] - 476s 304ms/step - loss: 0.7403 - accuracy: 0.7387 - val_

In [ ]:
# Final evaluation of the model 
scores = model.evaluate(X_test, y_test, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 81.26%
